# Chat-嬛嬛 v2.0 运行示例

### 一、对话集生成

In [ ]:
! bash ../../generation_dataset/generation.sh

### 二、模型微调

In [ ]:
! bash ../../fine_tune/lora/train.sh

### 三、通过代码调用微调模型

**ChatGLM**

In [1]:
from transformers import AutoModel, AutoTokenizer
import torch
from peft import PeftModel

# 本地底座模型参数路径
MODEL_PATH = "model path for ChatGLM"
# 微调参数路径
LoRA_PATH = "../../dataset/output"


/root/autodl-tmp/env/huanhuan-chat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 加载底座模型
model = AutoModel.from_pretrained(MODEL_PATH, trust_remote_code=True, device_map='auto')
# 加载 tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)
# 加载微调模型
model = PeftModel.from_pretrained(model, LoRA_PATH)

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 7/7 [00:12<00:00,  1.75s/it]


In [4]:
def get_completion(prompt, temprature=0, max_length=4096):
    with torch.no_grad():
        ids = tokenizer.encode(prompt)
        input_ids = torch.LongTensor([ids])
        out = model.generate(
            input_ids=input_ids,
            max_length=max_length,
            do_sample=False,
            temperature=temprature
        )
        out_text = tokenizer.decode(out[0])
        answer = out_text.replace(prompt, "").replace("\nEND", "").strip()
        return answer

In [5]:
prompt = "你是谁？"
print(get_completion(prompt))

/root/autodl-tmp/env/huanhuan-chat/lib/python3.10/site-packages/transformers/generation/utils.py:1452: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


我是甄嬛，家父是大理寺少卿甄远道。


**BaiChuan**

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

# 本地底座模型参数路径
MODEL_PATH = "model path for BaiChuan"
# 微调参数路径
LoRA_PATH = "../../dataset/output-baichuan"


/root/autodl-tmp/env/huanhuan-chat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 加载底座模型
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH, trust_remote_code=True).half().cuda()
# 加载 tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)
# 加载微调模型
model = PeftModel.from_pretrained(model, LoRA_PATH).half()

In [5]:
def get_completion(prompt, temprature=0, max_length=4096):
    with torch.no_grad():
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        out = model.generate(
            **inputs,
            max_length=max_length,
            do_sample=False,
            temperature=temprature
        )
        answer = tokenizer.decode(out[0][len(inputs.input_ids[0]):])
        return answer

In [6]:
prompt = "你是谁？"
print(get_completion(prompt))

 我是甄嬛，家父是大理寺少卿甄远道。</s>
